In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score, average_precision_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna

from functions import *

# Load your dataset
df = pd.read_csv("../data/clean/dataset.csv").fillna('')
X = df["lemmes"]
y = df["funny"]

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

2024-11-20 23:41:51.759947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732142511.779592   48530 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732142511.785396   48530 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 23:41:51.804223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
vectorizer = CountVectorizer(min_df=50, max_features=10000)

In [3]:
X_train_embeddings = vectorizer.fit_transform(X_train)
X_test_embeddings = vectorizer.transform(X_test)

In [4]:
# Train models
best_models = []

In [5]:
# Random Forest

rf_param_distributions = {
    "classifier__n_estimators": optuna.distributions.IntDistribution(50, 200),
    "classifier__max_depth": optuna.distributions.IntDistribution(3, 15),
    "classifier__min_samples_split": optuna.distributions.IntDistribution(2, 10),
    "classifier__min_samples_leaf": optuna.distributions.IntDistribution(1, 5),
}

rf_model, rf_acc = train_model_with_optuna(
    model=RandomForestClassifier(random_state=314),
    param_distributions=rf_param_distributions,
    X_train_embeddings=X_train_embeddings,
    y_train=y_train,
    X_test_embeddings=X_test_embeddings,
    y_test=y_test,
)
best_models.append(("random_forest", rf_model))

/home/michael/code/python/funnypress-data-study/notebooks/functions.py:139: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  search = OptunaSearchCV(
[I 2024-11-20 23:41:55,493] A new study created in memory with name: no-name-13a45776-8c8e-493c-ba54-1cf2629e3fa5
[I 2024-11-20 23:42:04,386] Trial 0 finished with value: 0.0 and parameters: {'classifier__n_estimators': 162, 'classifier__max_depth': 7, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 5}. Best is trial 0 with value: 0.0.
[I 2024-11-20 23:42:14,546] Trial 1 finished with value: 0.0 and parameters: {'classifier__n_estimators': 193, 'classifier__max_depth': 7, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 1}. Best is trial 0 with value: 0.0.
[I 2024-11-20 23:42:17,788] Trial 2 finished with value: 0.0 and parameters: {'classifier__n_estimators': 58, 'classifier__max_depth': 8, 'classifier__min_samples_split': 5, 'cl

Best parameters: {'classifier__n_estimators': 85, 'classifier__max_depth': 15, 'classifier__min_samples_split': 7, 'classifier__min_samples_leaf': 2}
Test Accuracy: 0.815605498991226
F1 Score: 0.012066365007541479
              precision    recall  f1-score   support

           0       0.82      1.00      0.90     17359
           1       1.00      0.01      0.01      3954

    accuracy                           0.82     21313
   macro avg       0.91      0.50      0.46     21313
weighted avg       0.85      0.82      0.73     21313



In [6]:
# XGBoost

xgb_param_distributions = {
    "classifier__n_estimators": optuna.distributions.IntDistribution(50, 200),
    "classifier__learning_rate": optuna.distributions.FloatDistribution(0.01, 0.3),
    "classifier__max_depth": optuna.distributions.IntDistribution(3, 10),
}

xgb_model, xgb_acc = train_model_with_optuna(
    model=XGBClassifier(random_state=314),
    param_distributions=xgb_param_distributions,
    X_train_embeddings=X_train_embeddings,
    y_train=y_train,
    X_test_embeddings=X_test_embeddings,
    y_test=y_test,
)
best_models.append(("xgboost", xgb_model))

/home/michael/code/python/funnypress-data-study/notebooks/functions.py:139: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  search = OptunaSearchCV(
[I 2024-11-20 23:44:23,803] A new study created in memory with name: no-name-39fc3ce4-7c10-44bc-a2dc-a44d3ad43add
[I 2024-11-20 23:44:26,281] Trial 0 finished with value: 0.3958100720296195 and parameters: {'classifier__n_estimators': 162, 'classifier__learning_rate': 0.10223368484312673, 'classifier__max_depth': 4}. Best is trial 0 with value: 0.3958100720296195.
[I 2024-11-20 23:44:29,791] Trial 1 finished with value: 0.5365765039231933 and parameters: {'classifier__n_estimators': 199, 'classifier__learning_rate': 0.2849341173889311, 'classifier__max_depth': 5}. Best is trial 1 with value: 0.5365765039231933.
[I 2024-11-20 23:44:31,206] Trial 2 finished with value: 0.29100310557835163 and parameters: {'classifier__n_estimators': 77, 'classifier__learning_rate': 0.025

Best parameters: {'classifier__n_estimators': 200, 'classifier__learning_rate': 0.29934142188815765, 'classifier__max_depth': 8}
Test Accuracy: 0.8815277060948716
F1 Score: 0.5870809484873263
              precision    recall  f1-score   support

           0       0.89      0.98      0.93     17359
           1       0.83      0.45      0.59      3954

    accuracy                           0.88     21313
   macro avg       0.86      0.72      0.76     21313
weighted avg       0.88      0.88      0.87     21313



In [7]:
# CatBoost
catboost_param_distributions = {
    "classifier__iterations": optuna.distributions.IntDistribution(50, 200),
    "classifier__learning_rate": optuna.distributions.FloatDistribution(0.01, 0.3),
    "classifier__depth": optuna.distributions.IntDistribution(3, 10),
}

catboost_model, catboost_acc = train_model_with_optuna(
    model=CatBoostClassifier(verbose=0, random_state=314),
    param_distributions=catboost_param_distributions,
    X_train_embeddings=X_train_embeddings,
    y_train=y_train,
    X_test_embeddings=X_test_embeddings,
    y_test=y_test,
)
best_models.append(("catboost", catboost_model))

/home/michael/code/python/funnypress-data-study/notebooks/functions.py:139: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  search = OptunaSearchCV(
[I 2024-11-20 23:45:44,295] A new study created in memory with name: no-name-caf5e402-0352-407b-8fed-73f0408eff8a
[I 2024-11-20 23:45:55,642] Trial 0 finished with value: 0.4093662591346961 and parameters: {'classifier__iterations': 162, 'classifier__learning_rate': 0.10223368484312673, 'classifier__depth': 4}. Best is trial 0 with value: 0.4093662591346961.
[I 2024-11-20 23:46:10,559] Trial 1 finished with value: 0.5539762274032631 and parameters: {'classifier__iterations': 199, 'classifier__learning_rate': 0.2849341173889311, 'classifier__depth': 5}. Best is trial 1 with value: 0.5539762274032631.
[I 2024-11-20 23:46:16,210] Trial 2 finished with value: 0.2986521241238643 and parameters: {'classifier__iterations': 77, 'classifier__learning_rate': 0.025252317480855793

Best parameters: {'classifier__iterations': 200, 'classifier__learning_rate': 0.29934142188815765, 'classifier__depth': 8}
Test Accuracy: 0.8822315019002487
F1 Score: 0.5990415335463258
              precision    recall  f1-score   support

           0       0.89      0.98      0.93     17359
           1       0.81      0.47      0.60      3954

    accuracy                           0.88     21313
   macro avg       0.85      0.72      0.77     21313
weighted avg       0.88      0.88      0.87     21313



In [8]:
# Stacking the best models
stacked_classifier = StackingClassifier(
    estimators=best_models,
    final_estimator=GradientBoostingClassifier(random_state=314),
    cv=5,
)


stacked_classifier.fit(X_train_embeddings, y_train)
y_pred = stacked_classifier.predict(X_test_embeddings)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Stacking Classifier Test Accuracy: {acc}")
print(f"Stacking Classifier F1 Score: {f1}")

print(classification_report(y_test, y_pred))


Stacking Classifier Test Accuracy: 0.888706423309717
Stacking Classifier F1 Score: 0.6578188113098673
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     17359
           1       0.77      0.58      0.66      3954

    accuracy                           0.89     21313
   macro avg       0.84      0.77      0.80     21313
weighted avg       0.88      0.89      0.88     21313

